# Key Methods in Physical Geography
# Computing Practical 3: Statistical Analysis of Geographic Data

This is a Jupyter notebook running on the University of Edinburgh Noteable Service (https://www.ed.ac.uk/information-services/learning-technology/noteable/about). Notebooks are interactive documents that can contain computer code (Python for this course), text and images.

For some components in this notebook, data files will be used. Any data file that is used for plotting with this and other notebooks needs to be uploaded to the Jupyter server -- unless they are provided for you, or there are commands provided to download the files directly to the Jupyter practical folder.

### Key Learning Outcomes

- More with NumPy arrays: Array Slicing and Reading from text files

- Creating and formatting histogram plots

- Working with Shapefiles and geodata frames in Python

    - Plotting shapefiles
    - Queries on shapefile data

- Statistics of Large Data sets

    - Simple descriptive statistics
    - String Formatting
    - Logical slicing
    - Lognormal data

## More with Numpy arrays

In this module you will learn how to create a histogram from a sample of data. However, before doing so we are going to go a bit further in terms of learning about NumPy arrays, since they are very central to Python: even if you do not use them directly, the libraries you use to do sophisticated analysis do. An important distinction between Python and GIS software is that GIS software is essentially a "black box" -- it is difficult to see its inner workings. For Python it is generally easier to "look under the hood".

We are going to create a histogram from data in the file `porosity.csv`: porosities (in %) from 105 soil samples. But before doing so we must read this data into Python. Last week we learned one way to do this, with the library `pandas` -- and we will use something similar later on today. But for now we will use a different way to load the data, a `numpy` function called `loadtxt`. At first it seems more complicated, but this allows us to have more control over what is read by Python.

In the cell below, import `numpy` as `np` and then type the command:

```python
pordata = np.loadtxt('porosity.csv', skiprows=1, delimiter=',')
```

And just so you can see something happened, type `print(pordata)` below.

### Optional Arguments

What took place here? `loadtxt` read text from a file `porosity.csv`, and saved it into the array variable `pordata` -- so from here we will use the variable `pordata` for analysis. You can view `porosity.csv` by clicking on the file in the Jupyter folder. If you do and compare with the output above, you will see that we did not read the first line, the line with column headers. This was achieved by providing `loadtxt` with the ***optional argument*** `skiprows=1`. In doing so we told Python to skip the first row. (<span style="color:blue">*read carefully! -- we will use more optional arguments when formatting histograms!*</span>)

It was also necessary to use the argument `delimiter=','`. A **delimiter** is how a program detects breaks between columns. Since .csv files have commas separating the columns, this is why a comma (in quotes, i.e. `','`) had to be the delimeter. 

If you run the cell above with either of these arguments missing you will see an error. Of course, this was more complicated than when we used `pandas` -- although with `pandas` it would be difficult to skip a prespecified number of rows.

In Python development optional arguments are called ***kwargs*** -- and though this is a silly name, you will see it often if you look at documentation for various Python functionalities.

### Array slicing

Now -- the first column of `porosity.csv` is sample number, we do not want or need this information. In the following cell, type the command

```python
porcolumn = pordata[:,1]
print(porcolumn)
```

The result we see is an array, `porcolumn`, containing only the porosity data, and not the sample number. What did `pordata[:,1]` do? This is an example of array **slicing**. Essentially, we said that we want to extract an array that contains *every row* of `pordata`, but only the `second` column. The ":" means that we want to extract **every row**. The "1" means that we want to extract the *second column*. (This is because Python numbers rows and columns the same way we number floors in buildings: the first column has index 0.)

A diagram of what we have done is shown below. The file contains data and row headers, but by skipping the first row we ignore the headers. Then slicing gives us data in the second column only.

**There was meant to be a figure embedded here, which generally works in Jupyter notebooks but Noteable will seemingly not allow it. Please click [here](https://www.geos.ed.ac.uk/~dgoldber/Untitled.png).**

If we wanted only the first 10 data rows, we would ask for `pordata[0:10,1]` or `pordata[:10,1]`. 

If we wanted only the last 10 data rows, we would ask for `pordata[-10:,1]`.

Reference on array slicing can be found [here](https://www.w3schools.com/python/numpy_array_slicing.asp). As a test, in the cell below see if you can extract only the 10th row, but *both columns*, into a variable `rowdata`??

## Creating and formatting histograms.

Now we are ready to plot a histogram. As with last week, before doing so you will need to 

```python
import matplotlib.pyplot as plt 
```

**NB**: as we learned before, you do *not* need to *alias* the library as `plt`. But it makes it easier -- e.g. in the following we can type `plt.hist` rather than `matplotlib.pyplot.hist` i.e. it is an abbreviation!

In the following cell, `import matplotlib.pyplot as plt`, and below this type 

```python
counts, bins, _ = plt.hist(porcolumn);
```

You will see above that a histogram of porosities has been created, but the number of bins and the bin colors have been decided for you. As with `np.loadtxt` you can pass to `plt.hist` *optional* arguments such as `color` and `bins`. Go back and modify the `hist` command to use a different number of bins (e.g. 12) and a different color (e.g. 'red' -- the color must be in quotes). Alternatively a full list of available  colors can be found [here](https://matplotlib.org/3.1.0/gallery/color/named_colors.html). 

Do you remember how to label axes? Label the x-axis `Porosity (%)`

You can also plot borders on the rectangles on the histogram with the optional argument `edgecolor='black'` and control their thickness with the optional argument e.g. `linewidth=2`.

There are many other options you can use, a complete list can be seen [here](https://matplotlib.org/3.3.2/api/_as_gen/matplotlib.pyplot.hist.html) but we do not have time to explore them all!

Note that the histogram command begins with `counts, _, _ = `. We did not need to include this to create the histogram i.e. the line could have read
```python
plt.hist(porcolumn)
```        
However many functions have **return variables** that might be useful. `counts` now contains an array of the count in each bin. Call `print(counts)` in the end of the cell above to see this information.

### Fitting to a normal distribution.

A *normal distribution* is a very common type of distribution used extensively in statistics (see Learn notes on statistics). **For statistical testing** (which is not emphasised in this course in favor of other techniques) it often is necessary to determine whether your data fits suitably to a normal distribution. A visual assessment can be done by plotting the normal distribution with the same *mean* $\mu$ and *variance* $\sigma$ as your data on top of the histogram. To remind you, the normal distribution is a function of the form

\begin{equation}
p(x) = \frac{1}{2\pi \sigma^2} e^{-\frac{1}{2}\frac{(x-\mu)^2}{\sigma^2}}
\end{equation}

But a library named `seaborn` provides a much simpler way to do this. Below the code uses the `seaborn.distplot` and `scipy.stats.norm` functions in order to do this. Note that it is more difficult this way to control the transparency of the histogram bars and their edge colors. Look carefully as you will use this function later for glacier area distributions.

**NB** there are more robust ways of determining whether distributions are normal. We will not explore them here but if you need to carry out "tests of normality" for your dissertation, they should be easy to locate by googling, or you can talk to your KMPG instructors.

In [ ]:
import seaborn as sns
from scipy import stats 
sns.distplot(porcolumn, fit=stats.norm, kde=False, color='slategray', bins=12)

## Working with Shapefiles: The Randolph Glacier Inventory

We will leave the porosity data behind and begin working with the types of data that physical geographers might use e.g. with GIS software to study the earth system.

The [Randolph Glacier Inventory](https://www.glims.org/RGI/) (RGI) is a collection of data on *all glaciers on Earth*. There are approximately 150,000 glaciers in the inventory, with a combined volume large enough to contribute 35 cm to global sea levels. The information is stored in *shape files*, which you saw in the context of QGIS -- we are going to use them in Python. 

The RGI is divided into 18 regions; we are going to download the data from two. As the files are large, you will not be asked to upload; rather, the following cell when run will download and unzip in your directory the data from 2 different regions: 11 (Central Europe) and 14 (South West Asia). These are *linux commands* to be executed by the cloud computer running Jupyter. Do not change them!!! You will see the download progress after you run it; when it is finished, *please check that you see additional files in your Jupyter practical folder* (there should be 5 files for each region). If you think something has gone wrong, talk to your demonstrator.

In [ ]:
! if [ -f 11_rgi60_CentralEurope.shp ]; then echo "file already downloaded"; else wget https://www.glims.org/RGI/rgi60_files/11_rgi60_CentralEurope.zip; unzip 11_rgi60_CentralEurope.zip; rm 11_rgi60_CentralEurope.zip; fi
! if [ -f 14_rgi60_SouthAsiaWest.shp ]; then echo "file already downloaded"; else wget https://www.glims.org/RGI/rgi60_files/14_rgi60_SouthAsiaWest.zip; unzip 14_rgi60_SouthAsiaWest.zip; rm 14_rgi60_SouthAsiaWest.zip; fi

You will notice in the output of the cell above that shapefiles (e.g. 14_rgi60_SouthAsiaWest.shp) have been downloaded. To use these files we will need to use the `geopandas` library. `geopandas` is similar to `pandas` but with more geographic functionality. In the following cell, let us read the RGI data into geopandas data frames (or *geodata frames*) by typing:

```python
import geopandas as gpd
gdfAsia = gpd.read_file('14_rgi60_SouthAsiaWest.shp')
gdfEur = gpd.read_file('11_rgi60_CentralEurope.shp')
```

Just like we explored data frames last practical, we can explore these geodataframes by typing e.g. `gdfEur` in the following cell and running. 

How many glaciers are there? Look at the columns of data. *RGIId* is the unique reference within the RGI -- notice that the first two digits after "RGI60-" give the region, 14. There is also area, longitude, latitude, height, and many other attributes; the last column, "Geometry", is a polygon defining each glacier boundary.



### Plottting Shapefiles

We can see the locations of all glaciers in a region by using the `plot()` function. This plot function is unique to `geopandas` and is distinct from `matplotlib.pyplot.plot` -- thus it needs to be referenced by the geodata frame. If we have a dataframe `gdf`, then we can plot it using this function. 

The cell below (with comments added) shows the location of the Central Europe glacier population within Europe. (It shows up as a little blue smudge where the alps are.) Note that it uses the shapefile `ne_10m_land.shp`, provided in your practical folder, which contains continental boundaries for the whole world. 

Can you modify this code to plot the Asian glaciers instead? You will need to change the longitude and latitude extents. (The South West Asia inventory stretches from 67 to 82 degrees East and about 30 to 40 degrees North, though you can experiment with ranges)

In [ ]:
gdfCont = gpd.read_file('ne_10m_land.shp')
# read the continental boundary shapefile

fig, ax = plt.subplots(figsize=(10,10))
# provide a set of figure axes on which to plot both shapefiles
# note that 'figsize' can change the size of the figure

gdfCont.plot(facecolor='yellow',edgecolor='k',ax=ax)
# plot continental outlines (with yellow shading over land)

plt.xlim(5,20)
# set longitude bounds from 5 to 20 degrees East
plt.ylim(38,48)
# set latitude bounds from 38 to 48 degrees North

gdfEur.plot(ax=ax)
# plot European glaciers


Something we did not do last practical is to filter a dataframe based on certain criteria. Let us do so now to look at one of the largest mountain glaciers in the world, Biafo glacier in Pakistan. The RGI Id of Biafo is **RGI60-14.00005**. 

We will produce a *new* geodata frame containing only this glacier. To do so, type:

```python
indBiafo = gdfAsia.RGIId.str.fullmatch('RGI60-14.00005')
gdfBiafo = gdfAsia[indBiafo]
```

There is a lot going on here so let's talk through it! In the first line, we are asking geopandas for all rows of `gdfAsia` such that the RGIId is exactly RGI60-14.00005. But to do this we have to reference the RGIId column (`gdf.RGIId`) and then the string associated with this column (`gdf.RGIId.str`) and than ask that it fully matches the string `'RGI60-14.00005'`). `indBiafo` then is a list of rows where this is true (there should be only one).

The next line creates a *new* geodata frame called `gdfBiafo` by extracting the rows from `gdfAsia` that satisfy the condition. As a result, gdfBiafo contains one 

This might seem very new -- but you will have a chance to practice similar operations below.

We can plot the outline of Biago glacier by itself now, by calling `gdfBiafo.plot()` similarly to above. Do so in the next cell. What does it look like? Does it look like what you would expect for a glacier? Can you identify the main branch and the tributaries?

Again, Python has chosen the color, and filled in the glacier outline. As with `hist`, we can pass optional arguments to modify this figure. The argument `facecolor="None"` will avoind filling in the outline, and the argument `edgecolor='black'` will hightlight the outline, though you can choose another color. The argument `figsize=(10,10)` will make the figure larger so we can see the detail (THough any two numbers can be chosen). Go back to the previous cell and provide these optional arguments. Can you see more detail now? 

Biafo is a very long glacier with many branching tributaries. The smaller tributaries are at higher elevations and experience mass gain due to snowfall. They feed into the wider, fast flowing trunk, where melting and fast flow take place.

### Overlying on rasters

It helps to see the glacier outline on context with the landscape. For this reason, in the cell below, code is provided that will do so. This code reads an elevation raster of the region surrounding Biafo (derived from Shuttle Radar Topography Mission data) and creates an image, and the shapefile geometry is then overlaid. We will cover raster plotting in a later practical, so we will not go into too much detail. Comments are provided in case you would like to make sense of what is done. Looking over and understanding commands will prepare you better for next practical.

In [ ]:
import rasterio as rio
# the library rasterio allows reading and processing of geotiff raster data.

dem_tif = rio.open('dem.tif')
# this command creates a link to the geotiff file dem.tif, but does not yet extract data.

dem = dem_tif.read(1).astype(float)
# this command extracts data from the geofiff file dem.tiff though the link dem_tif to an array called "dem"

fig, ax = plt.subplots(figsize=(10,10))
# this prepares a figure where we will plot both the raster and the shapefile

bounds = dem_tif.bounds
plt.imshow(dem,extent=(bounds.left,bounds.right,bounds.bottom,bounds.top)); 
# this command plots the raster file, with geographic extents matching the bounds of the raster

cbar = plt.colorbar()
cbar.set_label('elevation (m)')
# these lines create and label the colorbar

gdfBiafo_project = gdfBiafo.to_crs(dem_tif.crs)
# this is an important step. The shapefile has a different geodetic projection than the raster data, so the shapefile data 
# needs to be projected onto the new coordinates. a new geodata frame results.

gdfBiafo_project.plot(facecolor="None",edgecolor='k',linewidth = 1,ax=ax)
# finally, this is almost identical to your plotting command above; the difference is that you specify the figure axes ("ax") 
# on which to plot.

### Querying geodata frames based on numeric data

Above we filtered the RGI geodata frame based on *categorical* data i.e. the RGI ID. But we can filter geodata frames (and regular data frames!) based on numeric data as well. Filtering in this way is simpler. E.g. if we want to select Asian glaciers whose central Longitude is west of 79 E, we define a new geodata frame as follows (note `indTemp` and `gdfTemp` are my own variable names; use whatever you wish):
```python
indTemp = gdfAsia.CenLon < 79 
gdfTemp = gdfAsia[indTemp]
```
In the following cell use this and similar operations to answer:

- How many glaciers are there in Europe west of the Greenwich meridian (essentially the Pyrenees)?
- How many glaciers in Southwest Asia have area larger than 2 km?

To view the number in each extracted data frame, you can view the data frame as you did above with `gdfAsia`, or you can use the function `len()`. `len()` is a useful function that can be called with all sorts of variable types -- including data frames.

## Statistics of large data sets

It is useful to be able to carry out simple statistics on large sets of data such as the RGI, largely because it is so time consuming to examine each entry individually. 

We will extract columns from the geodata frames, as this will make it straightforward to carry out simple statistics and plot histograms. We will see how statistics of area vary between the two regions. Extracting the area column is a bit different than array slicing, but just as easy:

```python
areasAsia = gdfAsia.Area
areasEur = gdfEur.Area
```

These variables (`areasAsia` and `areasEur`) are not NumPy arrays like you worked with at the start of the practical but they are similar (they are `pandas` arrays). In fact, any functions expecting a NumPy array will accept a pandas array (you can see the proper name by typing `print(type(areasAsia))`, and we will use this fact to find some simple descriptive statistics of glacier areas.

### Simple descriptive statistics

Let's explore Python's simple descriptive statistics tools, and also take the chance to practice formatting Python *text* output. You will not often need to have "nice" looking text printed out in a Jupyter notebook, but you will quite often need to format figure captions, legends, and axis labels -- so it will come in handy.

The very simplest statistic of a sample is its size, found with the `np.size()` function. In the cell below, write the following in order to print out the size of the Asian glaciers sample:

```python
print( 'The size of Southwest Asia Glacier area inventory is: ' + str(np.size(areasAsia)) )
```

if you remember from the first practical, this results in a line of text composed of the phrase in quotes and the number of glaciers. The '+' *concatenates* (joins) this *string* with another string containing the number. **It is important to use the `str()` function to convert this number, `np.size(areasAsia)`, to a string**. A number is a different data type than a string, and trying to join a string and a number variable could cause an error (though sometimes is ok).

Below this line of code, do the same for the *maximum* (`np.max()`), *minimum* (`np.min()`), *mean* (`np.mean()`), *median* (`np.median()`), and *standard deviation* (`np.std`). Do the same for *skew* as well. (If you are not familiar with skew, please see the statistics document in the Learn folder, the last page in particular). NumPy **does not have** a *skew* function -- but this is part of the `scipy.stats` module, which you have already imported (i.e. you can call `stats.skew()`). 

A normally distributed sample will have median similar to its mean, and skew close to zero -- do you think the glacier area distribution is normal? 

**NOTE:** For `np.std()` there is an optional argument `ddof` which can be equal to either 0 or 1. If it is equal to 1, it yields the *sample* standard deviation (see the statistics document in the Learn folder). This option is used when investigating the **population** standard deviation from a small sample -- as it turns out the formula gives a better estimate. However, you are finding the standard deviation of the entire population, so you can ignore this optional argument which is 0 by default.

### More string formatting

The values printed above do not have units. The units of area in the RGI are given in km$^2$. Unfortunately we cannot use superscripts in output text (although we can in figure captions), so `km sq` will suffice. Can you edit the cell above so that the maximum is presented as:
```
The maximum of Southwest Asia Glacier area inventory is: 1077.958 km sq
```

Additionally you will see some of the statistics are given up to 16 decimal places. It is meaningless to give more decimal places than are given in the data set (3) and also it does not look nice. We can use the `round()` function of NumPy to round each number to no more than 3 decimal places. The `round()` function is called like this:

```python
newNum = np.round(Num, dec)
```

Here `Num` is a numeric variable, while `dec` is the number of desired decimal places, and `newNum` is a *new* numerical variable which is the result of rounding `Num`. That is, `np.round(3.33,1)` yields `3.3`. Modify your code in the cell above so that no values have more than 3 decimal places given. 

### Logical slicing

Let's say we want to know the average area of glaciers in Asia, but only **west** of 73 Degrees Longitude. We could actually select these rows from the geopandas data frame as we did earlier, but let's see a different way of doing this: *logical slicing*. Earlier we were introduced to `slicing` where we extract indices from an array in a *predefined* manner. We can also extract values from an array based on a *condition*. Let's see how this is done for the above example:

```python 
Lon = gdfEur.CenLon
areasEurWest = areasEur[Lon < 8]
print ('The mean of European Glaciers west of 8 Longitude is: ' + str(np.mean(areasEurWest)) + ' sq km')
```

Here, `Lon` is an array of centre longitudes for the glaciers. The following line then extracts the values of `areasEur` **only where the conditoin `Lon < 8` is satisfied** and saves this a new array, `areasEurWest`, and its mean is then found.

Try out this code in the following cell. Then, find the **standard deviation** of area of **Asian glaciers** above ___ m altitude. You will need to use the `Zmin` column -- the minimum elevation of each glacier. How does this compare with the average area you found above? Are you surprised by this?

### Lognormal Distributions.

As you did for porosity, plot a histogram of Asian glacier area with a normal distribution fitted. (Recall how you did this earlier; though you should not need to re-import `scipy.stats` or `seaborn`.) A nice feature of using the geopandas array `areasAsia` is that the x-axis will be labelled for you!!! (this is because of the type of the variable, `pandas.core.series.Series`, nothing specific to Asian glaciers.)

No matter how many bins you choose, the histogram will not look very nice. This is because the glacier areas are a very skewed distribution -- there are many, many small values and only a handful of large ones. A better way to visualise the distribution is through a **logarithmic transform**. We will find the histogram not of the areas but of the **base 10 logarithm** of the areas. This means that 0.01 km$^2$ becomes -2, and 1000 km$^2$ becomes 3 (we do not bother with units). Try defining

```python
logAreasAsia = np.log10(areasAsia)
```

and then creating a histogram as above with this new variable in the cell below. How does it look? Are the logarithmic values normally distributed? If the normal curve is a reasonable fit this is an example of a **lognormal distribution**.

Try choosing a sensible number of bins (i.e. not too blocky, but not too noisy). A sensible value is generally

\begin{equation}
\mathrm{number\ of\ bins} = \frac{\sqrt{\mathrm{size\ of\ sample}}}{\mathrm{range\ of\ sample}}
\end{equation}

where the range of the sample is the maximum minus the minimum. (The above must be rounded of course.)

The only drawback of the above figure is that the x-axis values are misleading as they are the logarithms of areas. We can fix this using `plt.xticks`. (this was used in the *extra material* at the end of last practical -- it was up to you to work throug this material!!!)

`plt.xticks` takes as arguments two lists: the values where we want to have tick marks on the x-axis, and the labels we want to have there. Thus if we want a tick at 10 km$^2$, we would use

```python
plt.xticks([1],[10])
```

As 1 is the *actual value* on the x-axis, but 10 is the area value that this corresponds to. Edit the above cell to place ticks at .01, .1, 1, 10, 100, and 1000 square km.